In [1]:
import re

from aocd import get_data

puzzle_input = get_data(day=7, year=2018).split('\n')
test_input = """Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.""".split('\n')

In [2]:
class Node:
    def __init__(self, name, is_test=False):
        self.name = name
        self.dependencies = []
        self.is_complete = False
        self.working = False
        self.complete_timestamp = None
        self.ts_offset = -60 if is_test else 0
        
    def start_work(self, timestamp):
        #print('Starting work on {} at {}'.format(self.name, timestamp))
        self.working = True
        self.complete_timestamp = timestamp + ord(self.name)-5 + self.ts_offset
        
    def step(self, timestamp):
        if self.working and timestamp >= self.complete_timestamp:
            #print('{} completed at {}'.format(self.name, timestamp))
            return self.complete()
    
    def add_dependency(self, dep):
        self.dependencies.append(dep)
        
    def can_complete(self):
        return not self.is_complete and all(dep.is_complete for dep in self.dependencies)
    
    def can_start(self):
        return not self.working and not self.is_complete and all(dep.is_complete for dep in self.dependencies)
    
    def complete(self):
        self.is_complete = True
        self.working = False
        return self.name
        
    def __repr__(self):
        return '<Node>: ' + self.name

In [3]:
def get_next_node(nodes):
    avail_nodes = []
    for key, node in nodes.items():
        if node.can_complete():
            avail_nodes.append(key)
            
    return min(avail_nodes) if avail_nodes else None

def get_next_threaded_node(nodes, threads=5):
    avail_nodes = []
    n_working = 0
    
    for key, node in nodes.items():
        if node.working:
            n_working += 1
            
        if node.can_start():
            avail_nodes.append(key)
            
    return sorted(avail_nodes)[:threads-n_working]

def build_nodes(puzzle_input, is_test=False):
    deps = [re.findall('tep (\w)', line) for line in puzzle_input]
    req_nodes = [dep[1] for dep in deps] + [dep[0] for dep in deps]
    nodes = {node:Node(node, is_test=is_test) for node in set(req_nodes)}
    for dep in deps:
        nodes[dep[1]].add_dependency(nodes[dep[0]])
        
    return nodes

def all_nodes_complete(nodes):
    return all(node.is_complete for node in nodes.values())

In [4]:
# Part 01

nodes = build_nodes(puzzle_input, is_test=False)

order = ''
while get_next_node(nodes):
    order += nodes[get_next_node(nodes)[0]].complete()
order

'EBICGKQOVMYZJAWRDPXFSUTNLH'

In [5]:
# Part 02

nodes = build_nodes(puzzle_input, is_test=False)

timestamp = 0
while True:
    avail_nodes = get_next_threaded_node(nodes, 5)
    for node in avail_nodes:
        nodes[node].start_work(timestamp)
    
    for node in nodes.values():
        node.step(timestamp)
        
    timestamp += 1
    
    if all_nodes_complete(nodes):
        break
        
print(timestamp)

906
